# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Scheuck  ->  M. Scheuck  |  ['M. Scheuck']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 81 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.08033


extracting tarball to tmp_2503.08033...

 done.
  0: tmp_2503.08033/methods.tex, 49 lines
  1: tmp_2503.08033/supplementary.tex, 58 lines
  3: tmp_2503.08033/main.tex, 759 lines
Retrieving document from  https://arxiv.org/e-print/2503.08196


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 3 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2503.08033/main.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'raw_methods' from 'tmp_2503.08033/raw_methods.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWa

...

 done.


R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Scheuck  ->  M. Scheuck  |  ['M. Scheuck']


Found 21 bibliographic references in tmp_2503.08523/T_CrA_MATISSE_arXiv.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.08670


extracting tarball to tmp_2503.08670...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.08196-b31b1b.svg)](https://arxiv.org/abs/2503.08196) | **Realising the potential of large spectroscopic surveys with machine-learning**  |
|| <mark>G. Guiglion</mark> |
|*Appeared on*| *2025-03-12*|
|*Comments*| *To appear in the proceedings of IAU Symposium 395 (4 pages, 1 figure); IAU Symposium 395 Proceedings, 2025*|
|**Abstract**|            Machine-learning is playing an increasing role in helping the astronomical community to face data analysis challenges, in particular in the field of Galactic Archaeology and large scale spectroscopic surveys. We present recent developments in the field of convolutional neural-networks (CNNs) for stellar abundances in the context of the Galactic spectroscopic surveys Gaia-ESO, and Gaia-RVS. Especially, by combining the full Gaia data product, we manage to characterize for the first time the [alpha/M] vs. [M/H] bimodality in the Galactic disc with Gaia-RVS spectra at low-S/N. This work is highly relevant for the next generation of large scale surveys such as MSE, 4MOST, and WST.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.08523-b31b1b.svg)](https://arxiv.org/abs/2503.08523) | **T CrA has a companion: First direct detection of T CrA B with VLTI/MATISSE**  |
|| J. Varga, et al. -- incl., <mark>R. v. Boekel</mark>, <mark>T. Henning</mark>, <mark>M. Scheuck</mark> |
|*Appeared on*| *2025-03-12*|
|*Comments*| *9 pages, 8 figures*|
|**Abstract**|            T CrA is a Herbig Ae-type young star in a complex circumstellar environment; it includes a circumstellar disk, accretion streamers, jets, and outflows. It has long been suspected to be a binary. However, until now, there has been no direct detection of a companion. Here we present new VLTI/MATISSE L- and N-band observations of T CrA taken between 2023 May and 2024 August with the aim of testing the binary nature of the system. We modeled the data with a geometric model using the Python tool oimodeler. We detected a companion (T CrA B) with a projected separation of $\Delta r = 153.2 \pm 1.2$ mas ($\approx 23$ au) toward the west direction at a position angle of $275.4 \pm 0.1^\circ$, in 2024 May-August. Our results support that the companion has a nearly edge-on orbit that is highly misaligned with respect to the circumprimary disk. Such a configuration could cause warping and tearing of the disk around the primary, which has been proposed by recent studies. In the L band the companion is extended, with a full width at half maximum (FWHM) size of $\sim 1$ au, suggesting that the emission comes from a disk around the secondary star. The companion flux is 0.2-0.3 Jy in the L band, and 0.2-0.7 Jy in the N band, accounting for 4-20% of the total emission at those wavelengths. The SED of the companion is compatible with thermal radiation of warm dust (600-800 K).         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.08033-b31b1b.svg)](https://arxiv.org/abs/2503.08033) | **A radio transient with unusually slow periodic emission**  |
|| N. Hurley-Walker, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-12*|
|*Comments*| *4 main figures, 9 extended figures*|
|**Abstract**|            The high-frequency radio sky is bursting with synchrotron transients from massive stellar explosions and accretion events, but the low-frequency radio sky has so far been quiet beyond the Galactic pulsar population and the long-term scintillation of AGN. The low-frequency band, however, is sensitive to exotic coherent and polarised radio emission processes such as electron cyclotron maser emission from flaring M-dwarfs [1], stellar magnetospheric plasma interactions with exoplanets [2], and a population of steep-spectrum pulsars [3], making Galactic plane searches a prospect for blind transient discovery. Here we report an analysis of archival low-frequency radio data that reveals a periodic, low-frequency radio transient. We find that the source pulses every 18.18 minutes, an unusual periodicity not previously observed. The emission is highly linearly polarised, bright, persists for 30--60 s on each occurrence, and is visible across a broad frequency range. At times the pulses comprise short ($<0.5$-s) duration bursts; at others, a smoother profile is observed. These profiles evolve on timescales of hours. By measuring the dispersion of the radio pulses with respect to frequency, we have localised the source to within our own Galaxy, and suggest that it could be an ultra-long period magnetar.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.08670-b31b1b.svg)](https://arxiv.org/abs/2503.08670) | **In Search of the Potentially Hazardous Asteroids in the Taurid Resonant Swarm**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2025-03-12*|
|*Comments*| *PSJ in press*|
|**Abstract**|            The Taurid Complex is a large interplanetary system that contains comet 2P/Encke, several meteoroid streams, and possibly a number of near-Earth asteroids. The size and nature of the system has led to the speculation that it was formed through a large-scale cometary breakup. Numerical investigations have suggested that planetary dynamics can create a resonant region with a large number of objects concentrated in a small segment of the orbit, known as the Taurid swarm, which approaches the Earth in certain years and provides favorable conditions to study the Taurid Complex. Recent meteor observations confirmed the existence of the swarm for mm- to m-sized objects. Here we present a dedicated telescopic search for potentially hazardous asteroids and other macroscopic objects in the Taurid swarm using the Zwicky Transient Facility survey. We determine from our non-detection that there are no more than 9--14 $H\leq24$ (equivalent to a diameter of $D\gtrsim100$~m) objects in the swarm, suggesting that the Encke--Taurid progenitor was $\sim10$~km in size. A progenitor of such a size is compatible with the prediction of state-of-the-art Solar System dynamical models, which expects $\sim0.1$ $D>10$~km objects on Encke-like orbits at any given time.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.08196/./figs/queiroz_plot.png']
copying  tmp_2503.08196/./figs/queiroz_plot.png to _build/html/
exported in  _build/html/2503.08196.md
    + _build/html/tmp_2503.08196/./figs/queiroz_plot.png
found figures ['tmp_2503.08523/./figs/T_CrA_2023-05-10T10_14_59_U1U2U3U4_L_TARGET_NOCHOP_FINALCAL_INT.png', 'tmp_2503.08523/./figs/T_CrA_2024-05-14T07_29_41_A0G2J2J3_L_TARGET_CHOPPED_4BCD_FINALCAL_INT.png', 'tmp_2503.08523/./figs/T_CrA_2024-06-02T04_30_13_U1U2U3U4_L_TARGET_CHOPPED_4BCD_FINALCAL_INT.png', 'tmp_2503.08523/./figs/model_image_3epochs.png', 'tmp_2503.08523/./figs/fit_2023-05-10_N_band.png', 'tmp_2503.08523/./figs/fit_2024-05-14_L_band.png', 'tmp_2503.08523/./figs/fit_2024-06-02_N_band.png', 'tmp_2503.08523/./figs/fit_2024-07-01_L_band.png', 'tmp_2503.08523/./figs/fit_2024-08-10_L_band.png', 'tmp_2503.08523/./figs/fit_2024-08-12_L_band.png']
copying  tmp_2503.08523/./figs/T_CrA_2023-05-10T10_14_59_U1U2U3U4_L_TARGET_NOCHOP_FINALCAL_INT.png to _build/html/
copying

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\zmax}{\langle Z_{\rm{max}} \rangle}$
$\newcommand{\eccentricity}{\langle e \rangle}$
$\newcommand{\aj}{AJ}$
$\newcommand{\araa}{ARA\&A}$
$\newcommand{\apj}{ApJ}$
$\newcommand{\apjl}{ApJ}$
$\newcommand{\apjs}{ApJS}$
$\newcommand{\apss}{Ap\&SS}$
$\newcommand{\aap}{A\&A}$
$\newcommand{\aapr}{A\&A~Rev.}$
$\newcommand{\aaps}{A\&AS}$
$\newcommand{\mnras}{MNRAS}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\pasj}{PASJ}$
$\newcommand{\qjras}{QJRAS}$
$\newcommand{\nat}{Nature}$
$\newcommand{\aplett}{Astrophys.~Lett.}$
$\newcommand{\aas}{AAS}$
$\newcommand{\eprint}{e--print}$</div>



<div id="title">

# Realising the potential of largespectroscopic surveys with machine-learning

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.08196-b31b1b.svg)](https://arxiv.org/abs/2503.08196)<mark>Appeared on: 2025-03-12</mark> -  _To appear in the proceedings of IAU Symposium 395 (4 pages, 1 figure); IAU Symposium 395 Proceedings, 2025_

</div>
<div id="authors">

<mark>G. Guiglion</mark>

</div>
<div id="abstract">

**Abstract:** Machine-learning is playing an increasing role in helping the astronomical community to face data analysis challenges, in particular in the field of Galactic Archaeology and large scale spectroscopic surveys. We present recent developments in the field of convolutional neural-networks (CNNs) for stellar abundances in the context of the Galactic spectroscopic surveys $*Gaia*$ -ESO, and $*Gaia*$ -RVS. Especially, by combining the full $*Gaia*$ data product, we manage to characterize for the first time the [ $\alpha$ /M ] vs. [ M/H ] bimodality in the Galactic disc with $*Gaia*$ -RVS spectra at low-S/N. This work is highly relevant for the next generation of large scale surveys such as MSE, 4MOST, and WST.

</div>

<div id="div_fig1">

<img src="tmp_2503.08196/./figs/queiroz_plot.png" alt="Fig1" width="100%"/>

**Figure 1. -** Two-dimensional histograms and contours of [$\alpha$/M] vs. [M/H] in 53 200 giants of the *Gaia*-RVS observed sample from \citet{Guiglion2024} with 15$<$S/N$<$25 and log(g)$<$2.2 within the training set limits. Stars are plotted in kiloparsec bins of galactocentric radius (R) and height above the Galactic plane (Z). (*fig:li*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.08196"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# T CrA has a companion$\thanks{Based on observations collected at the European Southern Observatory under ESO programmes 111.253T.001, 113.26PG.001, 113.26PG.006, and 113.26PG.008.}$: First direct detection of T CrA B with VLTI/MATISSE

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.08523-b31b1b.svg)](https://arxiv.org/abs/2503.08523)<mark>Appeared on: 2025-03-12</mark> -  _9 pages, 8 figures_

</div>
<div id="authors">

J. Varga, et al. -- incl., <mark>R. v. Boekel</mark>, <mark>T. Henning</mark>, <mark>M. Scheuck</mark>

</div>
<div id="abstract">

**Abstract:** T CrA is a Herbig Ae-type young star in a complex circumstellar environment; it includes a circumstellar disk, accretion streamers, jets, and outflows. It has long been suspected to be a binary. However, until now, there has been no direct detection of a companion. Here we present new VLTI/MATISSE $L$ - and $N$ -band observations of T CrA taken between 2023 May and 2024 August with the aim of testing the binary nature of the system. We modeled the data with a geometric model using the Python tool \texttt{oimodeler} .We detected a companion (T CrA B) with a projected separation of $\Delta r = 153.2 \pm 1.2$ mas ( $\approx23$ au) toward the  west direction at a position angle of $275.4 \pm 0.1◦$ , in 2024 May--August.Our results support that the companion has a nearly edge-on orbit that is highly misaligned with respect to the circumprimary disk. Such a configuration could cause warping and tearing of the disk around the primary, which has been proposed by recent studies. In the $L$ band the companion is extended, with a full width at half maximum (FWHM) size of $\sim1$ au, suggesting that the emission comes from a disk around the secondary star. The companion flux is $0.2$ -- $0.3$ Jy in the $L$ band, and $0.2$ -- $0.7$ Jy in the $N$ band, accounting for $4$ -- $20\%$ of the total emission at those wavelengths. The SED of the companion is compatible with thermal radiation of warm dust ( $600$ -- $800$ K).

</div>

<div id="div_fig1">

<img src="tmp_2503.08523/./figs/T_CrA_2023-05-10T10_14_59_U1U2U3U4_L_TARGET_NOCHOP_FINALCAL_INT.png" alt="Fig4.1" width="33%"/><img src="tmp_2503.08523/./figs/T_CrA_2024-05-14T07_29_41_A0G2J2J3_L_TARGET_CHOPPED_4BCD_FINALCAL_INT.png" alt="Fig4.2" width="33%"/><img src="tmp_2503.08523/./figs/T_CrA_2024-06-02T04_30_13_U1U2U3U4_L_TARGET_CHOPPED_4BCD_FINALCAL_INT.png" alt="Fig4.3" width="33%"/>

**Figure 4. -** Calibrated MATISSE $L$- and $LM$-band data sets with pipeline produced uncertainties. First column: Correlated flux as a function of the spatial frequency. Second column: Closure phase as a function of the spatial frequency corresponding to the longest baselines of the triangles. Third column: Single-dish flux as a function of the wavelength. Fourth column: $uv$ coverage of the observations. The correlated flux and the closure phase values are the average over the four non-chopped exposures of each observation, while the single-dish flux values are the average over the eight chopped exposures. The 2023 May 10 and 2024 June 2 data are UT observations, the rest are AT observations.  (*fig:data_L*)

</div>
<div id="div_fig2">

<img src="tmp_2503.08523/./figs/model_image_3epochs.png" alt="Fig1" width="100%"/>

**Figure 1. -** Best-fit model image corresponding to the fit to the three complete $L$-band data sets. The companion is clearly visible at $153$ mas separation toward the west direction. The J2000 coordinate of the central source is $\alpha = 19^\mathrm{h}01^\mathrm{m}58.79^\mathrm{s}$, $\delta = -36◦57$\arcmin$50.34$\arcsec$$. (*fig:model_img*)

</div>
<div id="div_fig3">

<img src="tmp_2503.08523/./figs/fit_2023-05-10_N_band.png" alt="Fig9.1" width="16%"/><img src="tmp_2503.08523/./figs/fit_2024-05-14_L_band.png" alt="Fig9.2" width="16%"/><img src="tmp_2503.08523/./figs/fit_2024-06-02_N_band.png" alt="Fig9.3" width="16%"/><img src="tmp_2503.08523/./figs/fit_2024-07-01_L_band.png" alt="Fig9.4" width="16%"/><img src="tmp_2503.08523/./figs/fit_2024-08-10_L_band.png" alt="Fig9.5" width="16%"/><img src="tmp_2503.08523/./figs/fit_2024-08-12_L_band.png" alt="Fig9.6" width="16%"/>

**Figure 9. -** Fits to the individual exposures in the second stage of our modeling. Each large three by four panel corresponds to a single epoch observation. A column in a panel shows the fits to a particular exposure, with the correlated flux on the top, closure phase in the middle, and single-dish flux at the bottom. The data are shown as solid lines, and the model as dashed lines. Correction factors are indicated in the legend of the correlated flux plots, for each baseline. The 2024 Aug 10 data set is incomplete, and thus only the closure phases were fitted. (*fig:fit_CF_CP*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.08523"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

361  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
